# Описание задачи
Датасет содержит релеавнтную информацию о пациетнах, на основании которой предполагается предсказать возможность возникновения инсульта.

## Описание признаков
1. id
2. Пол: "Мужчина", "Женщина" или "Другое"
3. Возраст
4. Гипертензия: 0, если у пациента нет гипертензии, 1 - иначе
5. Болезни сердца: 0, если не было, 1 - иначе
6. Когда либо был_а женат/замужем: "Нет" или "Да"
7. Вид деятельности: "ребёнок", "работает легально", "никогда не работал", "неофициально" или "самозанятый"
8. Где проживает: "сельская местность" или "город"
9. Средний баланс глюкозы в крови
10. Индекс массы тела
11. Зависимость от курения: "раньше курил", "никогда не курил", "курит" or "неизвестно"*
12. Приступ: 1, если приступ был, 0 - иначе
*"неизвестно" в зависимости от курения означает, что данная информация недоступна для конкретного пациента

Ссылка на датасет: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [83]:
import os
import numpy as np
import pandas
import matplotlib.pyplot
import seaborn

In [84]:
import sklearn
import sklearn.model_selection
import sklearn.neighbors
import sklearn.ensemble

In [85]:
import warnings
warnings.filterwarnings("ignore")

In [92]:
class Model:
    # Загрузка датасета
    def DataLoad(self):
        self.path = os.path.abspath(os.path.join(os.getcwd(), '.', 'healthcare-dataset-stroke-data.csv'))
        self.dataset = pandas.read_csv(self.path)
        self.dataset.columns = self.dataset.columns.str.strip()

    # Визуализация
    def DataVisual(self):
        print(self.dataset.describe())
        self.dataset.info()

    def fitPredict(self):
        x = self.dataset_normalized.drop(['stroke'], axis=1)
        y = self.dataset['stroke']
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = 0.25, random_state = 666)

        from sklearn import linear_model
        #regressor = linear_model.Ridge(alpha=.5)
        #regressor.fit(x_train, y_train)
        model = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 2)
        model.fit(x_train.values, y_train.values)

        y_pred_train = model.predict(x_train)
        y_pred = model.predict(x_test)

        from sklearn import metrics 
        print("Train: {}, Test: {}".format(sklearn.metrics.mean_squared_error(y_train, y_pred_train), sklearn.metrics.mean_squared_error(y_test, y_pred)))
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred)) 
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred)) 
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

    # Обработка датасета
    def filter(self):
        self.dataset.drop('id', axis=1, inplace=True)
        self.dataset['bmi'].replace({'N/A': None}, inplace=True)
        self.dataset['smoking_status'].replace({'Unknown': None}, inplace=True)
        self.dataset['bmi'].fillna(self.dataset['bmi'].median(), inplace=True)

        probabilities = self.dataset['smoking_status'].value_counts(normalize=True)
        missing = self.dataset['smoking_status'].isnull()
        self.dataset.loc[missing, 'smoking_status'] = np.random.choice(probabilities.index, size=len(self.dataset[missing]), p=probabilities.values)

        self.dataset['gender'] = np.where(self.dataset['gender'] == 'Female', 1, 0)
        self.dataset['ever_married'] = np.where(self.dataset['ever_married'] == 'Yes', 1, 0)
        self.dataset['Residence_type'] = np.where(self.dataset['Residence_type'] == 'Urban', 1, 0)

        self.dataset = pandas.get_dummies(self.dataset)

        rows_to_drop = self.dataset[
            (self.dataset['age'] < self.dataset['age'].quantile(0.25) - 1.5 * (self.dataset['age'].quantile(0.75) - self.dataset['age'].quantile(0.25))) 
            | (self.dataset['age'] > self.dataset['age'].quantile(0.75) + 1.5 * (self.dataset['age'].quantile(0.75) - self.dataset['age'].quantile(0.25)))].index

        self.dataset = self.dataset.drop(rows_to_drop)

        rows_to_drop = self.dataset[
            (self.dataset['hypertension'] < self.dataset['hypertension'].quantile(0.25) - 1.5 * (self.dataset['hypertension'].quantile(0.75) - self.dataset['hypertension'].quantile(0.25))) 
            | (self.dataset['hypertension'] > self.dataset['hypertension'].quantile(0.75) + 1.5 * (self.dataset['hypertension'].quantile(0.75) - self.dataset['hypertension'].quantile(0.25)))].index

        self.dataset = self.dataset.drop(rows_to_drop)

        rows_to_drop = self.dataset[
            (self.dataset['avg_glucose_level'] < self.dataset['avg_glucose_level'].quantile(0.25) - 1.5 * (self.dataset['avg_glucose_level'].quantile(0.75) - self.dataset['avg_glucose_level'].quantile(0.25))) 
            | (self.dataset['avg_glucose_level'] > self.dataset['avg_glucose_level'].quantile(0.75) + 1.5 * (self.dataset['avg_glucose_level'].quantile(0.75) - self.dataset['avg_glucose_level'].quantile(0.25)))].index

        self.dataset = self.dataset.drop(rows_to_drop)

        rows_to_drop = self.dataset[
            (self.dataset['bmi'] < self.dataset['bmi'].quantile(0.25) - 1.5 * (self.dataset['bmi'].quantile(0.75) - self.dataset['bmi'].quantile(0.25))) 
            | (self.dataset['bmi'] > self.dataset['bmi'].quantile(0.75) + 1.5 * (self.dataset['bmi'].quantile(0.75) - self.dataset['bmi'].quantile(0.25)))].index

        self.dataset = self.dataset.drop(rows_to_drop)
        self.dataset.drop('hypertension', axis=1, inplace=True)

        self.dataset_normalized = (self.dataset - self.dataset.mean(axis = 0)) / self.dataset.std(axis = 0)
        self.dataset_normalized.describe()

In [93]:
m = Model()

In [94]:
m.DataLoad()

In [95]:
m.filter()

In [96]:
m.DataVisual()

            gender          age  heart_disease  ever_married  Residence_type  \
count  4038.000000  4038.000000    4038.000000   4038.000000     4038.000000   
mean      0.592620    39.258366       0.034423      0.601535        0.508420   
std       0.491407    22.301675       0.182336      0.489643        0.499991   
min       0.000000     0.080000       0.000000      0.000000        0.000000   
25%       0.000000    21.000000       0.000000      0.000000        0.000000   
50%       1.000000    40.000000       0.000000      1.000000        1.000000   
75%       1.000000    56.000000       0.000000      1.000000        1.000000   
max       1.000000    82.000000       1.000000      1.000000        1.000000   

       avg_glucose_level          bmi       stroke  work_type_Govt_job  \
count        4038.000000  4038.000000  4038.000000         4038.000000   
mean           91.235037    27.449728     0.030956            0.123576   
std            22.224006     6.464669     0.173220       

In [97]:
m.fitPredict()

Train: 0.029392338177014532, Test: 0.033663366336633666
Mean Absolute Error: 0.033663366336633666
Mean Squared Error: 0.033663366336633666
Root Mean Squared Error: 0.18347579223601587
